In [ ]:
import quantz_repo
import pandas as pd
from pandas import DataFrame, Series
import tushare  as ts
import akshare as ak
from quantz_repo import utils,QuantzException
from quantz_repo.utils import df_2_mongo, mongo_2_df, yyyymmdd_2_int
from quantz_repo.model import BasicStockInfoItem, BasicTradingInfoItem, UsJoblessInitialClaimItem
from quantz_repo.models import MarketWidthItem
from quantz_repo.utils import get_next_day_in_YYYYMMDD, timestamp_2_YYYYMMDD, round_half_up,now_2_YYYYMMDD
from quantz_repo.utils import Fred
from quantz_repo import initialize_daily_trading_info, update_daily_trading_info_for,get_stock_basics, get_trade_dates_between,get_last_trade_date_in_ms_for
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from io import StringIO
import datetime

import mongoengine
mongoengine.connect('quantz')
# mongoengine.connect('quant_test', host='192.168.70.200', port=27017)

In [ ]:
rank_industry_level_at(level='L1',trade_date=1619625600000)

In [ ]:
rank_industry_level_at(level='L2',trade_date=1606060800000)

In [ ]:
rank_industry_level_at(level='L3',trade_date=1606060800000)

In [ ]:
rank_industry_level_between()

In [ ]:
rank_all_industry_between(since='20210427',end='20210506')

In [ ]:
quantz_repo.init_trade_calendar()

In [ ]:
quantz_repo.update_daily_trading_info()

In [ ]:
# quantz_repo.update_daily_trading_info()
quantz_repo.initialize_daily_trading_info()

In [ ]:
# update_daily_trading_info_for('000003.SZ')

In [ ]:
# 获取股票基本信息
quantz_repo.get_stock_basics()

In [ ]:
# 更定股票基本信息
quantz_repo.update_stock_basics()

In [ ]:
# 初始化行业分类
quantz_repo.initialize_industrial_classification()

In [ ]:
# 更新行业分类
quantz_repo.update_industry_classification()

In [ ]:
quantz_repo.rank_all_industry_between('20180101', '20210510')

In [ ]:
quantz_repo.rank_all_industry()

In [ ]:
quantz_repo.get_us_initial_jobless()

In [ ]:
quantz_repo.get_us_wei()

In [ ]:
quantz_repo.get_us_ccsa()

In [ ]:
import pandas as pd
import tushare as ts
from pandas import DataFrame
import datetime


PRICE_COLS = ['open', 'close', 'high', 'low', 'pre_close']
FORMAT = lambda x: '%.4f' % x

"""
def yyyymmdd_2_int(yyyymmdd: str) -> int:
    '''
    返回对应时间的毫秒值
    '''
    return int(datetime.datetime.strptime(yyyymmdd, '%Y%m%d').timestamp()) * 1000
"""

def batch_bar(ts_code='', start_date='', end_date='', freq='D', asset='E', adj=None):
    bars = ts.pro_bar(ts_code=ts_code, start_date=start_date, end_date=end_date, freq=freq, asset=asset)
    bars['freq']='D'
    if adj is not None:
        fcts = ts.pro_api().adj_factor(ts_code=ts_code, start_date=start_date, end_date=end_date)
        if fcts.shape[0] == 0:
            print('Failed to get adj factors')
            return
        result = DataFrame()
        for code in ts_code.split(','):
            code = code.strip()
            cur_bars = bars[bars['ts_code'] == code].reset_index(drop=True)
            cur_fcts = fcts[fcts['ts_code'] == code].reset_index(drop=True)[['trade_date', 'adj_factor']]
            if cur_bars.empty:
                print('No valid bar for %s between %s - %s, please check it' % (code,start_date, end_date))
                continue
            cur_bars = cur_bars.set_index('trade_date', drop=False).merge(cur_fcts.set_index('trade_date'), left_index=True, right_index=True, how='left')
            if 'min' in freq:
                cur_bars = cur_bars.sort_values('trade_time', ascending=False)
            cur_bars['adj_factor'] = cur_bars['adj_factor'].fillna(method='bfill')
            print('cur code:%s' % code)
            print('cur_bars')
            print(cur_bars)
            print('cur_fcts')
            print(cur_fcts)
            if adj is not None and cur_fcts.empty:
                print('Could not adj price for %s between %s-%s, cause adj factor empty' % (code, start_date, end_date))
            for col in PRICE_COLS:
                if adj == 'hfq' and not cur_fcts.empty:
                    cur_bars[col] = cur_bars[col] * cur_bars['adj_factor']
                if adj == 'qfq' and not cur_fcts.empty:
                    cur_bars[col] = cur_bars[col] * cur_bars['adj_factor'] / float(cur_fcts['adj_factor'][0])
                cur_bars[col] = cur_bars[col].map(FORMAT)
                cur_bars[col] = cur_bars[col].astype(float)
            cur_bars = cur_bars.drop('adj_factor', axis=1)
            if 'min' not in freq:
                cur_bars['change'] = cur_bars['close'] - cur_bars['pre_close']
                cur_bars['pct_chg'] = cur_bars['change'] / cur_bars['pre_close'] * 100
                cur_bars['pct_chg'] = cur_bars['pct_chg'].map(lambda x: FORMAT(x)).astype(float)
            else:
                cur_bars = cur_bars.drop(['trade_date', 'pre_close'], axis=1)
            cur_bars.reset_index(drop=True, inplace=True)
            result = pd.concat([result, cur_bars], ignore_index=True)
        bars = result.reset_index(drop=True)
        bars['trade_date'] = bars['trade_date'].map(yyyymmdd_2_int)
    print('batch bars')
    print(bars)
    return bars



# Sbars = batch_bar('603986.SH,000001.SZ,000002.SZ', start_date='20210520', end_date='20210522', adj='qfq')

In [ ]:
from quantz_repo.model import BasicTradingInfoItem
from quantz_repo.utils import df_2_mongo
import sys

def trading_info_2_mongo(info:DataFrame):
    if info is None or info.empty:
        print('Skip empty data')
        return
    info = info.sort_values(by='trade_date', axis=0,ascending=True,ignore_index=True)
    last = BasicTradingInfoItem.objects(ts_code=info['ts_code'][0], freq=info['freq'][0]).order_by('-trade_date').limit(1).first()
    start_date = last.trade_date if last is not None else -sys.maxsize-1
    print(info[info['trade_date'] > start_date])
    df_2_mongo(info[info['trade_date'] > start_date], BasicTradingInfoItem)
    print('done')
    return None
    


def update_daily_trading_info_in_batch(ts_code, start_date, end_date):
    print('gogogo')
    bars = batch_bar(ts_code=ts_code, start_date=start_date,end_date=end_date,adj='qfq')
    print(bars)
    if bars is None or bars.empty:
        print('None bar for %s between %s - %s' % (ts_code, start_date, end_date))
    print(bars)
    for code in ts_code.split(','):
        trading_info_2_mongo(bars[bars['ts_code']==code.strip()])
    print('zzzzz')
    return None

# update_daily_trading_info_in_batch('603986.SH,000001.SZ,000002.SZ', start_date='20210520', end_date='20210522')
# update_daily_trading_info_in_batch(ts_code='3300979.SZ,300980.SZ,300981.SZ,300982.SZ,300983.SZ,300985.SZ,300986.SZ,300987.SZ,300988.SZ,300989.SZ,300990.SZ,300991.SZ,300993.SZ,300999.SZ,600000.SH,600004.SH,600006.SH,600007.SH,600008.SH,600009.SH,600010.SH,600011.SH,600012.SH,600015.SH,600016.SH,600017.SH,600018.SH,600019.SH,600020.SH,600021.SH,600022.SH,600023.SH,600025.SH,600026.SH,600027.SH,600028.SH,600029.SH,600030.SH,600031.SH,600033.SH,600035.SH,600036.SH,600037.SH,600038.SH,600039.SH,600048.SH,600050.SH,600051.SH,600052.SH,600053.SH,600054.SH,600055.SH,600056.SH,600057.SH,600058.SH,600059.SH,600060.SH,600061.SH,600062.SH,600063.SH,600064.SH,600066.SH,600067.SH,600068.SH,600070.SH,600071.SH,600072.SH,600073.SH,600075.SH,600076.SH,600077.SH,600078.SH,600079.SH,600080.SH,600081.SH,600082.SH,600083.SH,600084.SH,600085.SH,600088.SH,600089.SH,600090.SH,600091.SH,600093.SH,600094.SH,600095.SH,600096.SH,600097.SH,600098.SH,600099.SH,600100.SH,600101.SH,600103.SH,600104.SH,600105.SH,600106.SH,600107.SH,600108.SH,600109.SH,600110.SH', start_date='20210510', end_date='20210519')

In [ ]:
from mongoengine import Document,LongField,StringField

class TradingInfoUpdateMetaItem(Document):
    # D W M Y H Min
    freq = StringField(required=True)
    # 上次成功更新的时间，以 ms 表示
    last_ok_date = LongField(required=True)

def get_last_ok_date(freq='D') -> str:
    last = TradingInfoUpdateMetaItem.objects(freq='D').order_by('-last_ok_date').limit(1).first()
    last_ms = last.last_ok_date if last is not None else get_last_trade_date_in_ms_for(exchange='SSE', adj=True)
    return timestamp_2_YYYYMMDD(last_ms)

def save_last_ok_date(trade_date, freq='D'):
    TradingInfoUpdateMetaItem(last_ok_date=yyyymmdd_2_int(trade_date), freq='D').save()

def update_all_daily_trading_info_in_batch():
    start_date = get_last_ok_date(freq='D')
    end_date = timestamp_2_YYYYMMDD(get_last_trade_date_in_ms_for(exchange='SSE', adj=True))
    if start_date is None or start_date == '' or end_date is None or end_date == '':
        print('Failed to update all daily trading info case invalid trade date(%s-%s)' % (start_date, end_date))
        return
    try:
        basics = quantz_repo.get_stock_basics()
        data = basics
        count = 0
        # Tushare限制最大100个
        batch_size = 100
        size = data.shape[0]
        done = False
        while not done:
            start = batch_size * count
            end = batch_size * (count + 1)
            if end > data.shape[0]:
                end = data.shape[0]
                done = True
            ts_codes = str(data['ts_code'][start:end].to_string(header=False, index=False))
            ts_codes = ts_codes.replace('\n', ',')
            # print(ts_codes)
            update_daily_trading_info_in_batch(ts_codes, start_date, end_date)
            print('####################')
            count = count + 1
    except Exception as e:
        print('Something is wrong' )
        raise QuantzException('Failed to update all daily trading info') from e
    else:
        save_last_ok_date(end_date, freq='D')
        print('Update daily trading info to %s ✔✔✔' % end_date)


update_all_daily_trading_info_in_batch()

In [ ]:
ts.pro_api().adj_factor(ts_code='689009.SH')

In [ ]:
TradingInfoUpdateMetaItem(last_ok_date=yyyymmdd_2_int('20210525'), freq='D').save()